### Instalando bibliotecas

In [ ]:
%%capture
pip install selenium

In [ ]:
%%capture
pip install xlrd

In [ ]:
%%capture
pip install geopy

In [2]:
import time
import json
import os
import unicodedata
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

# ==========================================
# CONFIGURAÇÃO DE BUSCA (INSIRA AQUI)
# ==========================================
CIDADE = "JardimOlinda"
ESTADO = "Parana"
CHECK_IN = "2026-03-10"  # Formato: AAAA-MM-DD
CHECK_OUT = "2026-03-15" # Formato: AAAA-MM-DD
# ==========================================

def format_location_for_url(cidade, estado):
    """Formata cidade e estado para o padrão do Airbnb (ex: Conde--Paraiba)."""
    def normalize(text):
        # Remove acentos e substitui espaços por hífens
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
        return text.strip().replace(" ", "-")
    
    cidade_fmt = normalize(cidade)
    estado_fmt = normalize(estado)
    return f"{cidade_fmt}--{estado_fmt}"

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def inject_dates_to_url(url, checkin, checkout):
    """Adiciona parâmetros de data à URL do Airbnb."""
    u = urlparse(url)
    query = parse_qs(u.query)
    query['check_in'] = [checkin]
    query['check_out'] = [checkout]
    new_query = urlencode(query, doseq=True)
    return urlunparse((u.scheme, u.netloc, u.path, u.params, new_query, u.fragment))

def scrape_airbnb_links():
    # Formatar a localização e construir a URL base
    loc_path = format_location_for_url(CIDADE, ESTADO)
    base_url = f"https://www.airbnb.com.br/s/{loc_path}/homes"
    
    # Injetar datas na URL
    target_url = inject_dates_to_url(base_url, CHECK_IN, CHECK_OUT)
    print(f"Iniciando busca para: {CIDADE}, {ESTADO}")
    print(f"URL de busca: {target_url}")
    
    driver = setup_driver()
    all_links = set()
    page_number = 1
    
    try:
        driver.get(target_url)
        time.sleep(5)
        
        while True:
            print(f"--- Processando página {page_number} ---")
            
            # Tentar fechar popups
            try:
                cookie_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Aceitar') or contains(text(), 'OK')]")
                cookie_button.click()
            except:
                pass

            # Rolar a página
            for _ in range(3):
                driver.execute_script("window.scrollBy(0, 1000);")
                time.sleep(1)
            
            # Seletores de links
            selectors = ['a[itemprop="url"]', 'div[data-testid="card-container"] a', 'a[href*="/rooms/"]']
            links_found = []
            for selector in selectors:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                if elements: links_found.extend(elements)
            
            if not links_found:
                print(f"Nenhum link encontrado na página {page_number}.")
                break

            current_page_new_links = 0
            for link in links_found:
                try:
                    href = link.get_attribute('href')
                    if href and '/rooms/' in href:
                        # Mantém as datas na URL do link individual
                        clean_url = href.split('?')[0] + f"?check_in={CHECK_IN}&check_out={CHECK_OUT}"
                        if clean_url not in all_links:
                            all_links.add(clean_url)
                            current_page_new_links += 1
                except:
                    continue
            
            print(f"Novos links nesta página: {current_page_new_links}. Total: {len(all_links)}")

            # Paginação
            try:
                next_button = None
                next_selectors = ['a[aria-label="Próximo"]', 'a[aria-label="Next"]', '//a[contains(@href, "section_offset")]']
                for sel in next_selectors:
                    try:
                        if sel.startswith('//'): next_button = driver.find_element(By.XPATH, sel)
                        else: next_button = driver.find_element(By.CSS_SELECTOR, sel)
                        if next_button: break
                    except: continue

                if next_button:
                    next_url = next_button.get_attribute('href')
                    driver.get(next_url)
                    page_number += 1
                    time.sleep(5)
                else:
                    print("Fim das páginas de resultados.")
                    break
            except:
                break

    finally:
        driver.quit()
    
    return list(all_links)

if __name__ == "__main__":
    links = scrape_airbnb_links()
    
    output_file = "airbnb_links_with_dates.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(links, f, indent=4, ensure_ascii=False)
    
    print(f"\nExtração concluída para {CIDADE}-{ESTADO}!")
    print(f"Total de links únicos extraídos: {len(links)}")
    print(f"Resultados salvos em: {os.path.abspath(output_file)}")

Iniciando busca para: JardimOlinda, Parana
URL de busca: https://www.airbnb.com.br/s/JardimOlinda--Parana/homes?check_in=2026-03-10&check_out=2026-03-15
--- Processando página 1 ---
Novos links nesta página: 18. Total: 18
--- Processando página 2 ---
Novos links nesta página: 4. Total: 22
Fim das páginas de resultados.

Extração concluída para JardimOlinda-Parana!
Total de links únicos extraídos: 22
Resultados salvos em: C:\Users\diego\Downloads\WebScraping\Airbnb\airbnb_links_with_dates.json


### Extrai dados dos anúncios

In [4]:
import time
import json
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from geopy.geocoders import Nominatim

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_number(text):
    if not text: return 0
    match = re.search(r'(\d+)', str(text))
    return int(match.group(1)) if match else 0

def main():
    input_file = "airbnb_links_with_dates.json"
    output_file = "airbnb_resultado_final_detalhado.csv"
    failed_links_file = "airbnb_links_falhos.json"
    
    if not os.path.exists(input_file):
        print(f"Erro: {input_file} não encontrado. Execute o script de extração de links primeiro.")
        return
    
    with open(input_file, "r") as f:
        room_links = json.load(f)

    driver = setup_driver()
    geolocator = Nominatim(user_agent="airbnb_final_exhaustive_scraper_v3")
    dados_coletados = []
    links_falhou = []

    # Processar links
    links_para_processar = room_links
    print(f"Iniciando extração detalhada de {len(links_para_processar)} anúncios...")

    for i, link in enumerate(links_para_processar):
        try:
            print(f"[{i+1}/{len(links_para_processar)}] Processando: {link}")
            driver.get(link)
            time.sleep(12) # Tempo para carregar dados dinâmicos
            
            # 1. Título
            try:
                titulo = driver.find_element(By.TAG_NAME, "h1").text
            except:
                titulo = "N/A"

            # 2. Capacidade (Hóspedes, Quartos, Camas, Banheiros) - Lógica Robusta
            hospedes = quartos = camas = banheiros = 0
            try:
                # O Airbnb costuma colocar isso em uma lista (ol/li) ou em uma div de visão geral
                # Vamos tentar capturar o texto de toda a seção de detalhes
                overview_selectors = [
                    'div[data-section-id="OVERVIEW_DEFAULT_V2"]',
                    'div[data-testid="listing-details-not-standalone"]',
                    'ol._199960j' # Classe comum para a lista de detalhes
                ]
                
                overview_text = ""
                for sel in overview_selectors:
                    try:
                        overview_text = driver.find_element(By.CSS_SELECTOR, sel).text
                        if overview_text: break
                    except: continue
                
                if not overview_text:
                    # Fallback: pegar o texto de todos os itens da lista
                    items = driver.find_elements(By.XPATH, "//ol/li")
                    overview_text = " ".join([it.text for it in items])

                # Regex em português para capturar os números
                h_m = re.search(r'(\d+)\s+hóspede', overview_text)
                q_m = re.search(r'(\d+)\s+quarto', overview_text)
                c_m = re.search(r'(\d+)\s+cama', overview_text)
                b_m = re.search(r'(\d+)\s+banheiro', overview_text)
                
                if h_m: hospedes = int(h_m.group(1))
                if q_m: quartos = int(q_m.group(1))
                if c_m: camas = int(c_m.group(1))
                if b_m: banheiros = int(b_m.group(1))
            except Exception as e:
                print(f"Erro ao extrair capacidade: {e}")

            # 3. Anfitrião e Tempo de Hospedagem
            anfitriao = tempo_hospedando = "N/A"
            try:
                host_section = driver.find_element(By.CSS_SELECTOR, 'div[data-section-id="HOST_OVERVIEW_DEFAULT"], div[data-testid="host-profile-container"]')
                host_text = host_section.text
                
                # Nome
                name_m = re.search(r'Anfitriã\(o\):\s*([^\n]+)', host_text)
                if name_m:
                    anfitriao = name_m.group(1).strip()
                else:
                    anfitriao = host_section.find_element(By.TAG_NAME, "h2").text.replace("Anfitriã(o): ", "").strip()
                
                # Tempo
                tempo_m = re.search(r'(\d+)\s+anos?\s+hospedando', host_text)
                if tempo_m:
                    tempo_hospedando = f"{tempo_m.group(1)} anos"
            except:
                pass

            # 4. Preços e Taxas
            preco_noite = preco_total = taxa_limpeza = noites = "N/A"
            try:
                price_widget = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="book-it-default"]')
                w_text = price_widget.text
                
                n_m = re.search(r'(R\$\s*[\d\.,]+)\s*[x/]', w_text)
                if n_m: preco_noite = n_m.group(1).strip()
                
                t_m = re.search(r'Total\s+(R\$\s*[\d\.,]+)', w_text)
                if t_m: preco_total = t_m.group(1).strip()
                
                nts_m = re.search(r'(\d+)\s+noites', w_text)
                if nts_m: noites = nts_m.group(1).strip()

                l_m = re.search(r'Taxa de limpeza\s+(R\$\s*[\d\.,]+)', w_text)
                if l_m: taxa_limpeza = l_m.group(1).strip()
            except:
                pass

            # 5. Coordenadas e Endereço
            lat = lng = endereco = "N/A"
            try:
                coords_match = re.findall(r'"latitude":([\d\.\-]+),"longitude":([\d\.\-]+)', driver.page_source)
                if coords_match:
                    lat, lng = coords_match[0]
                    location = geolocator.reverse((lat, lng), language='pt')
                    endereco = location.address if location else "N/A"
            except:
                pass

            dados_coletados.append({
                "Link": link,
                "Titulo": titulo,
                "Anfitriao": anfitriao,
                "Tempo_Hospedando": tempo_hospedando,
                "Preco_Noite": preco_noite,
                "Noites": noites,
                "Taxa_Limpeza": taxa_limpeza,
                "Preco_Total": preco_total,
                "Hospedes": hospedes,
                "Quartos": quartos,
                "Camas": camas,
                "Banheiros": banheiros,
                "Latitude": lat,
                "Longitude": lng,
                "Endereco": endereco
            })

        except Exception as e:
            print(f"⚠️ Falhou: {link} | Erro: {e}")
            links_falhou.append(link)

    # Salvar resultados
    df = pd.DataFrame(dados_coletados)
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    # Salvar links falhos
    if links_falhou:
        with open(failed_links_file, "w", encoding="utf-8") as f:
            json.dump(links_falhou, f, indent=4, ensure_ascii=False)
        print(f"Links falhos salvos em: {failed_links_file}")

    print(f"\nSucesso! {len(dados_coletados)} registros salvos em {output_file}")
    driver.quit()

if __name__ == "__main__":
    main()

Iniciando extração detalhada de 22 anúncios...
[1/22] Processando: https://www.airbnb.com.br/rooms/49849816?check_in=2026-03-10&check_out=2026-03-15
[2/22] Processando: https://www.airbnb.com.br/rooms/1251464272667900504?check_in=2026-03-10&check_out=2026-03-15
[3/22] Processando: https://www.airbnb.com.br/rooms/1140777804386657404?check_in=2026-03-10&check_out=2026-03-15
[4/22] Processando: https://www.airbnb.com.br/rooms/1049331626593801941?check_in=2026-03-10&check_out=2026-03-15
[5/22] Processando: https://www.airbnb.com.br/rooms/1210307411814531128?check_in=2026-03-10&check_out=2026-03-15
[6/22] Processando: https://www.airbnb.com.br/rooms/1140408984128530658?check_in=2026-03-10&check_out=2026-03-15
[7/22] Processando: https://www.airbnb.com.br/rooms/1537691166562164008?check_in=2026-03-10&check_out=2026-03-15
[8/22] Processando: https://www.airbnb.com.br/rooms/799463764801477935?check_in=2026-03-10&check_out=2026-03-15
[9/22] Processando: https://www.airbnb.com.br/rooms/143775404